# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.

%load_ext dotenv
%dotenv 



In [2]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [12]:
import os
from glob import glob

# Write your code below.

# Read PRICE_DATA path:
PRICE_DATA = os.getenv('PRICE_DATA')
print("PRICE_DATA path is:", PRICE_DATA)

# Find all PARQUET files path under PRICE_DATA directory:
parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive=True)
print("PARQUET path files are:", parquet_files)


PRICE_DATA path is: ../../05_src/data/prices/
PARQUET path files are: ['../../05_src/data/prices\\ACN\\ACN_2001\\part.0.parquet', '../../05_src/data/prices\\ACN\\ACN_2001\\part.1.parquet', '../../05_src/data/prices\\ACN\\ACN_2002\\part.0.parquet', '../../05_src/data/prices\\ACN\\ACN_2002\\part.1.parquet', '../../05_src/data/prices\\ACN\\ACN_2003\\part.0.parquet', '../../05_src/data/prices\\ACN\\ACN_2003\\part.1.parquet', '../../05_src/data/prices\\ACN\\ACN_2004\\part.0.parquet', '../../05_src/data/prices\\ACN\\ACN_2004\\part.1.parquet', '../../05_src/data/prices\\ACN\\ACN_2005\\part.0.parquet', '../../05_src/data/prices\\ACN\\ACN_2005\\part.1.parquet', '../../05_src/data/prices\\ACN\\ACN_2006\\part.0.parquet', '../../05_src/data/prices\\ACN\\ACN_2006\\part.1.parquet', '../../05_src/data/prices\\ACN\\ACN_2007\\part.0.parquet', '../../05_src/data/prices\\ACN\\ACN_2007\\part.1.parquet', '../../05_src/data/prices\\ACN\\ACN_2008\\part.0.parquet', '../../05_src/data/prices\\ACN\\ACN_2008\\pa

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [18]:
# Write your code below.

dd_px = dd.read_parquet(parquet_files).set_index("ticker") # create the dask dataframe

dd_feat = (
    dd_px
    .groupby('ticker', group_keys=False)
    .apply(
        lambda x: x.sort_values('Date', ascending=True)
                  .assign(
                      Close_lag_1 = x['Close'].shift(1),
                      Adj_Close_lag_1 = x['Adj Close'].shift(1),
                      returns = (x['Close'] / x['Close'].shift(1)) - 1,
                      hi_lo_range = x['High'] - x['Low']
                  ),
        meta={
            'Date': 'datetime64[ns]',
            'Open': 'f8',
            'High': 'f8',
            'Low': 'f8',
            'Close': 'f8',
            'Adj Close': 'f8',
            'Volume': 'i8',
            'source': 'object',
            'Year': 'int32',
            'Close_lag_1': 'f8',
            'Adj_Close_lag_1': 'f8',
            'returns': 'f8',
            'hi_lo_range': 'f8'
        }
    )
)
dd_feat


,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range
npartitions=90,,,,,,,,,,,,,
ACN,datetime64[ns],float64,float64,float64,float64,float64,int64,object,int32,float64,float64,float64,float64
ALDX,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,...,...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,...,...,...,...,...,...,...,...,...,...,...,...,...


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [20]:
# Write your code below.

# Convert dask dataframe into panda:
df_feat = dd_feat.compute()

# Sort the data by ticker and date:
df_feat = df_feat.sort_values(['ticker', 'Date'])

# Add a 10-day moving average of returns per ticker:
df_feat['returns_ma_10days'] = df_feat.groupby('ticker')['returns'].transform(lambda x: x.rolling(10).mean())
df_feat



,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range,returns_ma_10days
ticker,,,,,,,,,,,,,,
ACN,2001-07-19,15.10,15.29,15.00,15.17,11.404394,34994300.0,ACN.csv,2001,NaN,NaN,NaN,0.930000,NaN
ACN,2001-07-20,15.05,15.05,14.80,15.01,11.284108,9238500.0,ACN.csv,2001,113.099998,106.026108,0.003714,1.559998,NaN
ACN,2001-07-23,15.00,15.01,14.55,15.00,11.276587,7501000.0,ACN.csv,2001,113.519997,106.419830,-0.007223,1.389999,NaN
ACN,2001-07-24,14.95,14.97,14.70,14.86,11.171341,3537300.0,ACN.csv,2001,112.699997,105.651115,0.021384,2.070000,NaN
ACN,2001-07-25,14.70,14.95,14.65,14.95,11.238999,4208100.0,ACN.csv,2001,115.110001,107.910400,0.003041,0.850006,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,2020-03-26,4.06,4.53,3.88,4.51,4.510000,1668500.0,ZIXI.csv,2020,4.000000,4.000000,0.127500,0.650000,-0.000807
ZIXI,2020-03-27,4.49,4.71,4.10,4.60,4.600000,1146800.0,ZIXI.csv,2020,4.510000,4.510000,0.019956,0.610000,0.004262
ZIXI,2020-03-30,4.83,4.87,4.44,4.64,4.640000,1212000.0,ZIXI.csv,2020,4.600000,4.600000,0.008696,0.430000,0.015491


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

Converting to pandas made calculating the 10-day moving average simple, but wasn’t strictly necessary. Using Dask would be better for very large datasets that don’t fit in memory, though rolling calculations across partitions are more complex. For small-to-medium data, pandas is easier; for large-scale production, Dask is safer.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.